# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [21]:


from __future__ import division
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
import warnings
import math
warnings.filterwarnings('ignore')
%matplotlib inline

In [4]:
data = pd.io.stata.read_stata('us_job_market_discrimination.dta')

In [5]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [6]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [12]:
data.shape

(4870, 65)

In [18]:
# keep only the interesting columns

df = pd.concat([data.race,data.call], axis=1)
#print('sample size =',len(data))
df.head()
#df.shape

,race,call
0,w,0.0
1,w,0.0
2,b,0.0
3,b,0.0
4,w,0.0


In [43]:
df.shape

(4870, 2)

In [32]:
w = df[df.race=='w']
b = df[df.race=='b']
#print(w)

## Qestion no 1: 

* **What test is appropriate for this problem? Does CLT apply?**

Two-sample t-test is appropiate because we are trying to determine if two population means are equal.</br>
CLT applies because our sample size is large enough, where n > 30 (n = 4870). And the observations are independent.

## Question no 2: 

* **What are the null and alternate hypotheses?**

**Null Hypothesis:** The mean difference between the callback rates for black & white-sounding names is zero. </br>

**Alternative Hypothesis:** There is a significant (non-zero) difference between callback rates for black & white-sounding names.



## Question no 3:

* **Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.**

In [46]:
# black sample size
n_b = len(data[data['race'] == 'b']) 

# white sample size
n_w = len(data[data['race'] == 'w'])

# black callback rate
p_b = sum(data[data['race'] == 'b'].call)/n_b 

# white callback rate
p_w = sum(data[data['race'] == 'w'].call)/n_w 

# conduct two-sample t-test
p = (n_b*p_b+n_w*p_w)/(n_b+n_w) # joint callback
std = math.sqrt(p*(1-p)*((1/n_b)+(1/n_w))) # joint standard deviation

test_stat = (p_b-p_w)/std
p_val = stats.t.sf(abs(test_stat), df=len(data)-1)*2

print('test statistic:',test_stat)
print('p value:',p_val)

test statistic: -4.108412152434346
p value: 4.0493178875903686e-05


In [47]:
# black standard error
stderr1 = math.sqrt((p_b*(1-p_b))/n_b) 

# white standard error
stderr2 = math.sqrt((p_w*(1-p_w))/n_w) 

# standard error for difference
stderr_diff = math.sqrt(math.pow(stderr1,2) + math.pow(stderr2,2)) # standard error for difference

# calculate confidence interval for difference in the distributions
conf_int_lower = abs(p_b-p_w)-1.96*stderr_diff
conf_int_upper = abs(p_w-p_w)+1.96*stderr_diff
conf_int = [conf_int_lower,conf_int_upper]

print('confidence interval for the difference of black & white callback rate:')
print(conf_int)

confidence interval for the difference of black & white callback rate:
[0.016777447859559147, 0.015255406349886438]


So what we see is, the p-value = 4.04 e-5 and 95% confidence interval = [0.017, 0.047] for the difference between the two.

## Question no 4:

* **Write a story describing the statistical significance in the context or the original problem.**

As we have seen, that the p-value is less than 0.05 and the confidence interval does not contain 0. We can definitely conclude that, there is a statistically significant difference between the callback rates of black and white-sounding names.

## Question no 5:
* **Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?**

No, the conclusion above does not imply that the race is the most important factor in callback success. Actually, We have not analyzed any other features like education, years of experience etc. which sound more promising towards the callback success. We need to consider them in our study in order to conclude the *most important* factor.